# Setup

1. Import python3 libaries
1. Download the Kaggle dataset on stanford-car-parks
1. Set up the files for experiments


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm,tqdm_notebook
import pandas as pd
from pathlib import Path
import json
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects
import matplotlib.pyplot as plt
import scipy.io as sio

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

import time
import os
import PIL.Image as Image
from IPython.display import display


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jutrera/stanford-car-dataset-by-classes-folder")
PATH = path
print("Path to dataset files:", path)

print(f"PATH : {PATH}")

! rm -rf  ~/content/
! mkdir ~/content/
! mv -v {PATH}/car_data/*   ~/content/



Path to dataset files: /home/ngailam_ho/.cache/kagglehub/datasets/jutrera/stanford-car-dataset-by-classes-folder/versions/2
PATH : /home/ngailam_ho/.cache/kagglehub/datasets/jutrera/stanford-car-dataset-by-classes-folder/versions/2
mv: cannot stat '/home/ngailam_ho/.cache/kagglehub/datasets/jutrera/stanford-car-dataset-by-classes-folder/versions/2/car_data/*': No such file or directory


In [3]:
! ls -l /home/ngailam_ho/content

total 0


# Training

classes_path = "/root/.cache/kagglehub/datasets/jutrera/stanford-car-dataset-by-classes-folder/versions/2/car_data/car_data"

train_annos_path = classes_path
test_annos_path = classes_path
#classes_path = 'devkit/cars_meta.mat' names.csv
#classes_path = "/root/.cache/kagglehub/datasets/jutrera/stanford-car-dataset-by-classes-folder/versions/2/car_data/car_data"

def get_labels(annos_path, classes_path):
    car_annos = sio.loadmat(path + annos_path)
    car_meta = sio.loadmat(path + classes_path)
    annotations = car_annos["annotations"][0,:]
    nclasses = len(car_meta["class_names"][0])
    class_names = dict(zip(range(1,nclasses),[c[0] for c in car_meta["class_names"][0]]))

    labelled_images = {}
    dataset = []
    for i,arr in enumerate(annotations):
        # the last entry in the row is the image name
        # The rest is the data, first bbox, then classid
        dataset.append([y[0][0] for y in arr][0:5]+[arr[5][0]])
    # Convert to a DataFrame, and specify the column names
    temp_df = pd.DataFrame(dataset,
                      columns =['BBOX_X1','BBOX_Y1','BBOX_X2','BBOX_Y2','ClassID','filename'])

    temp_df = temp_df.assign(ClassName=temp_df.ClassID.map(dict(class_names)))
    temp_df.columns = ['bbox_x1','bbox_y1','bbox_x2','bbox_y2','class_id','filename', 'class_name']
    return temp_df

## setup training

In [ ]:
def train_model(model, criterion, optimizer, scheduler, n_epochs = 5):
    
    losses = []
    accuracies = []
    test_accuracies = []
    # set the model to train mode initially
    model.train()
    model=model.to('cuda')
    for epoch in range(n_epochs):
        since = time.time()
        running_loss = 0.0
        running_correct = 0.0
        for i, data in enumerate(trainloader, 0):

            # get the inputs and assign them to cuda
            inputs, labels = data
            
            inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
             # forward + backward + optimize
                
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # calculate the loss/acc later
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()

        epoch_duration = time.time()-since
        epoch_loss = running_loss/len(trainloader)
        epoch_acc = 100/32*running_correct/len(trainloader)
        print("Epoch %s, duration: %d s, loss: %.4f, acc: %.4f" % (epoch+1, epoch_duration, epoch_loss, epoch_acc))
        
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        model.eval()
        test_acc = eval_model(model)
        test_accuracies.append(test_acc)
        
        # re-set the model to train mode after validating
        model.train()
        scheduler.step(test_acc)
        since = time.time()
    print('Finished Training')
    return model, losses, accuracies, test_accuracies


In [6]:
def eval_model(model):
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            images, labels = data
            
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model_ft(images)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        test_acc = 100.0 * correct / total
    print('Accuracy of the network on the test images: %0.2f %%' % (test_acc))
    return test_acc


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using CUDA : {device}")
print(torch.cuda.get_device_name(device))



dataset_dir = "../StanfordCars/data/stanford-cars-dataset/data/car_data/car_data/"

train_tfms = transforms.Compose([transforms.Resize((400, 400)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomRotation(15),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
test_tfms = transforms.Compose([transforms.Resize((400, 400)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

print(f"root : {dataset_dir}/train ")
dataset = torchvision.datasets.ImageFolder(root = f"{dataset_dir}/",
                                           transform = train_tfms)
trainloader = torch.utils.data.DataLoader(dataset, batch_size = 32,
                                          shuffle=True, num_workers = 2)

dataset2 = torchvision.datasets.ImageFolder(root =  f"{dataset_dir}/",
                                            transform = test_tfms)
testloader = torch.utils.data.DataLoader(dataset2, batch_size = 32,
                                         shuffle=False, num_workers = 2)



Using CUDA : cuda:0
NVIDIA GeForce RTX 2080 Ti
root : ../StanfordCars/data/stanford-cars-dataset/data/car_data/car_data//train 


In [8]:
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 196)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=0.01,momentum=0.9)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold=0.9)

model_ft, training_losses, training_accs, test_accs = train_model(model_ft, criterion, optimizer, lrscheduler, n_epochs=10)

##################################################################

nb_classes = 196

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in tqdm_notebook( enumerate(testloader)):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)


/home/ngailam_ho/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ngailam_ho/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, duration: 291 s, loss: 0.6814, acc: 60.5463
Accuracy of the network on the test images: 62.52 %
Epoch 2, duration: 294 s, loss: 0.5910, acc: 62.6238
Accuracy of the network on the test images: 63.13 %
Epoch 3, duration: 294 s, loss: 0.5784, acc: 62.5133
Accuracy of the network on the test images: 63.08 %
Epoch 4, duration: 295 s, loss: 0.5669, acc: 62.6459
Accuracy of the network on the test images: 63.38 %
Epoch 5, duration: 295 s, loss: 0.5558, acc: 62.8006
Accuracy of the network on the test images: 63.50 %
Epoch 6, duration: 295 s, loss: 0.5240, acc: 63.5962
Accuracy of the network on the test images: 63.95 %
Epoch 7, duration: 295 s, loss: 0.5167, acc: 63.4017
Accuracy of the network on the test images: 64.10 %
Epoch 8, duration: 295 s, loss: 0.5117, acc: 63.9675
Accuracy of the network on the test images: 64.19 %
Epoch 9, duration: 295 s, loss: 0.5072, acc: 63.8879
Accuracy of the network on the test images: 64.21 %
Epoch 10, duration: 294 s, loss: 0.5029, acc: 64.3343
A

/tmp/ipykernel_3767748/3978349732.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, (inputs, classes) in tqdm_notebook( enumerate(testloader)):


0it [00:00, ?it/s]

tensor([[1881.,  740., 5429.,  ...,    0.,    0.,    0.],
        [ 137., 2159., 4219.,  ...,    0.,    0.,    0.],
        [1890.,  766., 5394.,  ...,    0.,    0.,    0.],
        ...,
        [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
        [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
        [   0.,    0.,    0.,  ...,    0.,    0.,    0.]])


In [9]:
#### SAVE MODEL
torch.save(model_ft.state_dict(), 'Rasnet50.pth')

#model_json = model_ft.to_json()
#with open("model_arch.json", "w") as json_file:
#    json_file.write(model_json)
# model = YourModel()  # Create an instance of your model
# model.eval()  # Set the model to evaluation mode
# model_ft = models.resnet50(pretrained=True)
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 196)
# model.load_state_dict(torch.load('model_ft.pth'))

In [10]:
###############################################################

In [11]:
! ls -l Rasnet50*

-rw-rw-r-- 1 ngailam_ho ngailam_ho 95955971 Nov 24 23:26 Rasnet50.pth
